根據把首頁往下滑，發現網站會對server送一個GET請求下一個30篇文章
- 2018(G) https://ithelp.ithome.com.tw/articles/10203216
- 2019(G) https://tlyu0419.github.io/2019/04/06/Crawl-Dcard/
- 2020 https://blog.jiatool.com/posts/dcard_api_v2/

### API root

- 看板資訊 http://dcard.tw/_api/forums
- 文章資訊 http://dcard.tw/_api/forums/{看板名稱}/posts
- 文章內文	http://dcard.tw/_api/posts/{文章編號}
- 文章內引用連結	http://dcard.tw/_api/posts/{文章編號}/links
- 文章內留言	http://dcard.tw/_api/posts/{文章編號}/comments

### API wirte
- 文章數量 https://www.dcard.tw/_api/posts?limit=100
- 文章分頁 http://dcard.tw/_api/posts/?before=文章id (before 回傳由新到舊 after舊到新)
- 文章留言 http://dcard.tw/_api/posts/文章id/comments
- 留言分頁 http://dcard.tw/_api/posts/文章id/comments?after=floor數字
- 熱門文章 https://dcard.tw/_api/posts?popular=true

### others
文章資訊(meta) 與 文章內留言 預設使用熱門度 (popularity) 作為排序，而且一次請求 (request) 中只回應 30 筆。
* 想要讓這兩項使用 時間 排序，可在 GET 參數中加入 popular=false
* 欲取得更多的 文章資訊(meta)，可以使用 before={某文章編號}來獲得早於 #{某文章編號} 的另外 30 筆 資訊。
* 欲取得更多的 文章內留言，可以使用 after={某樓層} 來獲得大於 #{某樓層} 的另外 30 筆 留言。

In [1]:
import requests
import json
import pandas as pd    # 引用套件並縮寫為 pd
from bs4 import BeautifulSoup
# https://www.dcard.tw/f/makeup

GET 美妝版首頁，Use BeautifulSoup with html, wrtie a txt

 

In [2]:
p = requests.Session()                             # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/f/makeup")  # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
#sel

#for link in sel:
#    a.append(link["href"])
    #print('https://www.dcard.tw'+link.get('href'))

In [3]:
# slope the title
def Catch(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [4]:
# KeyError: 'id'
# Oh！文章不見了 這篇文章已被作者刪除。 按這裡 看更多文章吧
# https://www.dcard.tw/f/makeup/p/235079399
# Catch(235079399)

In [5]:
# # 1w : 234227720
# # 2w : 233254532
# # 3w : 232166760
# # 4w : 231148015
# # 5w : 230149776
# url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=233254532" 

In [8]:
import random
import time

#url = "http://dcard.tw/_api/forums/makeup/posts?popular=false&limit=1"
url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=230149776" 
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(Catch(rejs[0]['id']),ignore_index=True) 
#df
print(str(int(df['id'].tail(1))) )

230149668


In [9]:
count = 0
delay_choices = [3, 5, 7]  # delay second
delay = random.choice(delay_choices) 

for j in range(300):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    print(last)
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        count = count+1
        print('data reading...',count,'id:',rejs[i]['id'],rejs[i]['title'])
    delay = random.choice(delay_choices)
    time.sleep(delay)  
df.to_csv('data/Dcard_makeup.csv', index=False,encoding = "UTF-8")
print(df.shape)
df

230149668
data reading... 1 id: 230149639 #黑眼圈顏色
data reading... 2 id: 230149544 粉撲發霉？？
data reading... 3 id: 230149373 使用遮瑕後很乾
data reading... 4 id: 230149331 1028冬季特賣會
data reading... 5 id: 230149318 #問 etvos純礦粉底
data reading... 6 id: 230149146 指甲慘慘慘～～～
data reading... 7 id: 230149134 #圖 愛用妝品綜合分享（附今日妝容）
data reading... 8 id: 230149016 平民版嘖嘖稱齊！
data reading... 9 id: 230148783 #分享#圖  高姿態約會妝髮分享💄👠
data reading... 10 id: 230148360 #分享 雅詩蘭黛幸福聖誕獻禮 抽到正品
data reading... 11 id: 230148337 原來是安麗....ಥ_ಥ
data reading... 12 id: 230148265 返鄉投票戰利品之超冷門 倩碧 超聚光無暇BB氣墊粉餅好用嗎？
data reading... 13 id: 230148263 理膚寶水保濕化妝水跟外用A酸
data reading... 14 id: 230148130 The ordinary 甘醇酸 包裝蓋腐蝕
data reading... 15 id: 230148122 #情報 Maybeline
data reading... 16 id: 230147894 乾肌換季後的空空賞+冷門MIT香水
data reading... 17 id: 230147540 隱形眼鏡保養液使用習慣問卷
data reading... 18 id: 230147451 #NARS #週慶特惠 #專業底妝組
data reading... 19 id: 230147274 Addiction遮瑕盤x兩步驟痘痘遮瑕💪🏻
data reading... 20 id: 230147079 #問 Elf磨砂唇膏(薄荷)
data reading... 21 id: 230146851 

data reading... 166 id: 230124204 超過30樣的空空賞❣彩妝/清潔/護唇/髮品
data reading... 167 id: 230123982 #寶雅 #Nail Holic
data reading... 168 id: 230123980 What’s in my makeup bag 🥰
data reading... 169 id: 230123938 #分享 博士倫renu保養液新包裝也太夢幻
data reading... 170 id: 230123928 #圖 品木宣言精油唇膏與潤唇膏
data reading... 171 id: 230123646 #情報 heme六色眼影盤（嘉義）
data reading... 172 id: 230123571 #圖#新手眼影求批
data reading... 173 id: 230123475 essence新品
data reading... 174 id: 230123464 跟著give盤教學，用had盤畫眼妝！
data reading... 175 id: 230123417 擦防曬乳真的很娘砲？
data reading... 176 id: 230123201 網美心大爆發 eos小開箱
data reading... 177 id: 230122988 韓系美妝Peripera唇彩、innisfree防曬乳、LANEIGE晚安唇膜心得
data reading... 178 id: 230122951 2018 年末美妝空空賞☃️趴ONE
data reading... 179 id: 230122463 #閒聊 那些你覺得好用卻被邊緣的好物？？
data reading... 180 id: 230122379 #雅漾 敏弱肌體驗組免費索取
data reading... 181 id: 230122203 #試色 romand zero gram唇膏(偷渡實測BRONX+1028遮瑕)
data reading... 182 id: 230122140 健康膚色的眼影刷色❤️
data reading... 183 id: 230122043 耳洞位置
data reading... 184 id: 230121878 光療 美甲分享
data r

data reading... 329 id: 230107166 讓你欲罷不能的豆沙紅棕Pat McGrath #flesh5
data reading... 330 id: 230107141 #問 colourpop
data reading... 331 id: 230106903 ＃圖 男友家過夜 素顏見真章之雅詩金三角篇
data reading... 332 id: 230106827 Nail holic、Honey suckle指甲油💅分享
data reading... 333 id: 230106782 週年慶跟朋友合買東西會算信用卡回饋%數嗎(FRIDAY錢包)
data reading... 334 id: 230106643 #請益 眼影盤選擇
data reading... 335 id: 230106468 連鎖店做臉
data reading... 336 id: 230106416 到底在工三小😑😑😑😑😑
data reading... 337 id: 230106346 #分享 不斷回購的雅聞單品
data reading... 338 id: 230106121 有人喜歡用不防水的睫毛膏嗎
data reading... 339 id: 230106026 我在台灣，我不鼓勵不專業的言論
data reading... 340 id: 230105911 #問 治療粉刺的藥
data reading... 341 id: 230105870 Selfridges聖誕節優惠碼來囉
data reading... 342 id: 230105799 #innisfree新品 #氣墊腮紅 #超美漸層刷具
data reading... 343 id: 230105761 情報 *更 11/21特賣，不只到十點 —Tokyo dolls的蝦皮限時特賣🔥
data reading... 344 id: 230105730 #分享 換季乾癢擦yuskin ai藍瓶止癢有效
data reading... 345 id: 230105619 論點反動物實驗？！論點是要求進行人道的動物實驗吧？
data reading... 346 id: 230105338 #分享 Sephora八折買了什麼💸💸
data reading... 347 i

data reading... 490 id: 230089433 innisfree指甲油
data reading... 491 id: 230089372 #黑心代購 勿忘Vaxxa事件
data reading... 492 id: 230089285 10支專櫃唇膏分享 #試色
data reading... 493 id: 230088882 #情報 黑色星期五折扣整理
data reading... 494 id: 230088821 😍燒爆😍Dashing diva美甲片
data reading... 495 id: 230088755 黑特！大黑特！板橋霧眉態度非常差！（未消費）
data reading... 496 id: 230088546 #問 新買的essence眼影發霉？
data reading... 497 id: 230088531 今年秋冬土色唇彩討論區
data reading... 498 id: 230088526 #試色 catrice060土色腮紅跟其他不專業臉部彩妝心得分享
data reading... 499 id: 230088443 Flormar #DC29
data reading... 500 id: 230088376 #試色 露華濃REVLON #535
230088376
data reading... 501 id: 230088241 #圖 #試色 這植感，挺有質感( a.k.a.雙十一剁手開箱😎)
data reading... 502 id: 230088239 #請益保養品使用順序
data reading... 503 id: 230088206 15個腮紅試色（只有手上試色）
data reading... 504 id: 230088016 防曬的重要！
data reading... 505 id: 230087931 被小紅書燒到的稚優泉眼影
data reading... 506 id: 230087748 #請益 etude house 粉底液金蓋色號
data reading... 507 id: 230087743 怎麼製造空靈感
data reading... 508 id: 230087662 痘痘肌困擾……😭
data reading... 509 id: 23

data reading... 654 id: 230073750 Get ready with me！上學妝容
data reading... 655 id: 230073666 #問粉底 #BB粉底簡單分享
data reading... 656 id: 230073647 我的延禧攻略指甲
data reading... 657 id: 230073610 #中壢sogo週年慶戰利品 NARS篇
data reading... 658 id: 230073540 #週年慶 #開箱 好姊妹互相傷害~SABON白茶🌿+暮光森林🎄
data reading... 659 id: 230073513 #有选手 打亮选择
data reading... 660 id: 230073253 🔗#圖 What’s in my bag?返鄉化妝包🔗
data reading... 661 id: 230072920 有人有去用過臉部整骨嗎
data reading... 662 id: 230072657 🍁秋冬唇膏🍁
data reading... 663 id: 230072578 Penhaligon's潘海利根Luna聖誕禮盒!
data reading... 664 id: 230072278 真真假假眉毛雨衣
data reading... 665 id: 230072265 Sephora 8折分享❤️
data reading... 666 id: 230072251 光療 美甲 分享 蛋頭、三眼怪、毛怪
data reading... 667 id: 230072004 海昌隱眼求解❤️
data reading... 668 id: 230071986 問 美材行的雙眼皮貼
data reading... 669 id: 230071830 #請益 選擇眼影盤
data reading... 670 id: 230071699 剁手週年慶🔪
data reading... 671 id: 230071675 詢問 蝦皮代購賣家
data reading... 672 id: 230071647 nail holic 指甲油分享
data reading... 673 id: 230071554 #問 潤色護唇膏
data reading... 674 id

data reading... 817 id: 230058837 好大隻的擦擦筆😂
data reading... 818 id: 230058767 你們每天化妝嗎
data reading... 819 id: 230058694 畫眉神器
data reading... 820 id: 230058686 nars英式搖滾6色頰彩盤開箱🤤🤤
data reading... 821 id: 230058657 去角質後續保養問題
data reading... 822 id: 230058529 Pat McGrath 新色mini唇膏組 試色
data reading... 823 id: 230058528 我在週年慶敗了我半個月月薪
data reading... 824 id: 230058212 男生妝容分享👀 #更照
data reading... 825 id: 230058077 四隻Dior唇彩分享
data reading... 826 id: 230058069 這是一篇男友看到會殺人的鬼打牆刷色
data reading... 827 id: 230057802 濃濃秋冬🍁巧克力色Loreal小棕皮#642
data reading... 828 id: 230057759 👀10、11月眼影分享
data reading... 829 id: 230057729 ＃圖 美翻天的雅詩蘭黛楊冪唇膏組
data reading... 830 id: 230057535 #問 屈臣士自有品牌的護唇膏
data reading... 831 id: 230057392 混油皮今冬の真愛 碧兒泉晶透奇蹟水➕青春無敵水凝露
data reading... 832 id: 230057358 那些年的母湯唇膏💄( ･᷄ὢ･᷅ )更#試色
data reading... 833 id: 230057296 #求助 VDL持久粉底液 兩罐差異
data reading... 834 id: 230057017 #圖 霧面季節🍁BobbiBrown金緻極霧唇膏💄唇釉 五支試色
data reading... 835 id: 230056720 #圖 Dior四支最愛唇膏-超惹火/藍星/潤唇膏
data reading... 836 id: 23005665

data reading... 973 id: 230042544 Heme 六色眼影盤
data reading... 974 id: 230042542 黑人問號的消費者
data reading... 975 id: 230042471 週年慶我買了什麼(mac.ipsa.three.addiction)
data reading... 976 id: 230042344 口服a酸副作用
data reading... 977 id: 230042165 口服A酸7個月 沮喪分享(慎入)
data reading... 978 id: 230042082 #圖 美翻GA胖胖瓶506～媽咪的久違自拍💄
data reading... 979 id: 230042046 請益 Gelovery的產品
data reading... 980 id: 230041962 #問#情報 肌研卵肌化妝水價錢
data reading... 981 id: 230041893 #購物分享#週慶撒錢記💸（四）
data reading... 982 id: 230041820 Jo malone 牡丹與胭脂麂絨、葡萄柚、青檸羅勒與柑橘 分享！
data reading... 983 id: 230041662 sugarfree指甲油分享
data reading... 984 id: 230041534 elsagusa產品
data reading... 985 id: 230041505 Pony霧光氣墊粉餅讓我大過敏
data reading... 986 id: 230041418 ✨PAT MCGRATH LABS打亮盤
data reading... 987 id: 230041366 #靠櫃 專櫃粉底液
data reading... 988 id: 230041155 霓淨思雙11優惠＆分享我的購物車
data reading... 989 id: 230041032 蘭蔻零粉感選色
data reading... 990 id: 230040707 #2更 OPT鉑金灰 無濾鏡分享
data reading... 991 id: 230040587 #情報 豐台灣面膜3件169
data reading... 992 id: 230040572 #隱眼 愛爾

data reading... 1133 id: 230028698 近期喜歡的愛用品分享🐸
data reading... 1134 id: 230028687 #圖 再添一枚😍 GA小紅氣墊專屬刻字版入宮！
data reading... 1135 id: 230028682 #情報 Bobbi Brown美國官網8折
data reading... 1136 id: 230028653 愛用唇膏分享
data reading... 1137 id: 230028602 求推薦 開架#南瓜#土橘色唇膏
data reading... 1138 id: 230028557 #求推薦 混合肌冬天保濕
data reading... 1139 id: 230028449 #情報 1028app 11元搶購
data reading... 1140 id: 230028321 ＃圖 擦了好秋的雅詩絕對慾望奢華美唇露＃307
data reading... 1141 id: 230028288 請益，男生香水推薦
data reading... 1142 id: 230028266 【情報】康是美11/10一日限定89折優惠
data reading... 1143 id: 230028176 Colourpop 眼影盤分享✨ #文長 #刷色
data reading... 1144 id: 230028147 L’Oréal 小棕皮#291
data reading... 1145 id: 230028139 #請益MAC唇膏真偽判別
data reading... 1146 id: 230028116 更# 用wet n wild rose in the air 畫微煙燻妝（etude house 十色眼影盤使用心得😂小雷
data reading... 1147 id: 230028097 超美唇色的抉擇V.S曖昧對象
data reading... 1148 id: 230028081 擦這個毛真的會變多變濃密嗎...?
data reading... 1149 id: 230028033 澳洲緹莉護手霜
data reading... 1150 id: 230027799 #分享 CLIO光感無瑕氣墊粉餅 柔霧版
data reading... 1151 id:

data error... 84 J: 12
data reading... 1285 id: 230014460 #情報 嘉丹妮爾45折🛍️
data reading... 1286 id: 230014425 #圖 #分享 既優雅又卡哇依 日本美甲
data reading... 1287 id: 230014405 #內雙的日常 很多雜談的紅色系咖啡色眼妝偽教程
data reading... 1288 id: 230014394 #極推大推一生推！雅漾安敏保濕水凝乳
data reading... 1289 id: 230014362 kiko奶油唇膏 #105#114 分享
data error... 89 J: 12
data reading... 1290 id: 230014262 #問 #Cimeosil喜美而疤痕凝膠
data reading... 1291 id: 230014170 想請問蘭蔻這兩款防曬差別在哪？
data reading... 1292 id: 230014103 #分享 我的光療❤️
data reading... 1293 id: 230014077 #分享 那些我的愛用唇膏 💄part2
data reading... 1294 id: 230014032 男友變女友😆
data reading... 1295 id: 230014021 #問 想知道各位平均一天都帶妝多久？
data reading... 1296 id: 230013985 #圖＃分享 橘色mix粉色的秋冬妝容🍂
data reading... 1297 id: 230013779 #問#台中#紋唇
data reading... 1298 id: 230013708 提煉再濃縮2-極簡旅行保養包(全開架)
data reading... 1299 id: 230013492 救救睫毛
data reading... 1300 id: 230013479 #分享1028隱藏關卡中獎
230013479


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# ---------------正式版本分隔線---------------
- 測試版本代號：1.2  2020
- 正式版本代號：2.03  2021
- 開發人員： EGG Jerry Three


dva.x_x 版本開發

In [ ]:
#找到 dav.x_x 發表過的文章

p = requests.Session()                             # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/@dva.x_x")  # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
sel
a=[]
for link in sel:
    a.append(link["href"])
    print(link.get('href')[-9:])
    print('https://www.dcard.tw'+link.get('href'))

In [ ]:
# slope the title
def CatchUser(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'department':rejs['department'],  #for author
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'department', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'department':"",    #for author
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'department', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [ ]:
# 熱門美妝版測試 (100筆)
url = 'https://www.dcard.tw/@dva.x_x'
#url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=5'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(CatchUser(rejs[i]['id']),ignore_index=True)   #0 - 29
print(df.shape)
#df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色
# 235066991

In [ ]:
import random
import time

url = "http://dcard.tw/_api/forums/makeup/posts?popular=false&limit=1"
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(CatchUser(rejs[0]['id']),ignore_index=True) 
#df
print(str(int(df['id'].tail(1))) )

count = 0
delay_choices = [1, 3, 5]  # delay second
delay = random.choice(delay_choices) 

for j in range(1):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    print(last)
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        count = count+1
        print('data reading...',count,'id:',rejs[i]['id'],rejs[i]['title'])
    delay = random.choice(delay_choices)
    time.sleep(delay)  
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

In [ ]:
# slope the title
def CatchPost(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [ ]:
#CatchPost(234999065)
#url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
url = 'https://www.dcard.tw/_api/posts/235103419'
resq = requests.get(url)
rejs = resq.json()
rejs

In [ ]:
url = 'https://www.dcard.tw/_api/posts/234912821'
resq = requests.get(url)
rejs = resq.json()
rejs

In [ ]:
# url = 'https://www.dcard.tw/_api/posts/235103302'
resq = requests.get(url)
rejs = resq.json()
rejs

# --------------- dva.x_x 版本分隔線 ---------------
- 測試版本代號：Jerry 1.0
- 開發人員： Jerry


In [ ]:
#test = open("data/test.txt","w",encoding='UTF-8')

In [ ]:
# asb
import random
import time

url = "http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=99"
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(Catch(rejs[0]['id']),ignore_index=True)   #0 - 29

delay_choices = [1]#, 3, 5, 7,20]  #延遲的秒數
delay = random.choice(delay_choices)  #隨機選取秒數

for j in range(10):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=true&limit=99&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 101 == 0:
            print('第',j,'次執行，data reading...',i,'last',last,'chocice',delay)
    delay = random.choice(delay_choices)  #隨機選取秒數
    time.sleep(delay)  #延遲

df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

In [ ]:
# 熱門美妝版測試 (100筆)
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']),ignore_index=True)   #0 - 29
    if i % 20 == 0:
        print('data reading...',i)
print(df.shape)
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色
# 235066991

In [ ]:
#last = str(int(df['ID'].tail(1))) #last dt
#last

In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df['ID'].tail(1))) # 找出爬出資料的最後一筆ID
    url = 'https://www.dcard.tw/_api/posts?popular=true&before=' + last
    #url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 10 == 0:
            print('第',j,'次執行，data reading...',i)
            
print(df.shape)
df

In [ ]:
last = str(int(df['ID'].tail(1)))
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df

In [ ]:
# 熱門美妝版測試 (100筆)

for i in range(10):
    last = str(int(df['ID'].tail(1)))
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    df = pd.DataFrame()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']),ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
    print(df.shape)

df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df

# 99	235031315	#請益 #問 LA girl 遮瑕挑色

In [ ]:
last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID '235031315'
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)


df

In [ ]:

#df

In [ ]:
##ok ^
for j in range(3):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)
#df

In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
print(df.shape)
df


In [ ]:
df.to_excel('data/Dcard_markeup.excel', index=False)

欄位	說明	備註
ID	編號	
title	標題	
content	內容	
excerpt	摘要	
createdAt	發布時間	
updatedAt	更新時間	
commentCount	留言數	
forumName	分類	中文
forumAlias	分類	英文
gender	性別	
likeCount	心情數量	
reactions	心情細項	把以上心情細分為「愛心」、「哈哈」、「跪」、「森77」、「驚訝」等類型
topics	標籤	

In [ ]:
# 固定碼測試
def Catch_test(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )
Catch_test(235020702)